In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker
import pandas as pd

# Import models from your models.py file
from scripts.models import Base, Flight, Airline, Airport, Route

In [2]:
# Load environment variables
load_dotenv()

# Database connection settings - adjust these as needed
DB_URL = os.getenv("DATABASE_URL")
API_KEY = os.getenv("AVIATIONSTACK_API_KEY")
engine = create_engine(DB_URL, echo=True)  # Set echo=True to see SQL statements


# API Configuration
BASE_URL = "https://api.aviationstack.com/v1/"
MAX_RETRIES = 3
RETRY_DELAY = 2  # seconds

ModuleNotFoundError: No module named 'psycopg2'

In [3]:
# from sqlalchemy import text

# # Define the tables to drop
# #tables_to_drop = ['airlines', 'airports', 'routes', 'flights']
# tables_to_drop = ['flights']

# # Ensure you have the engine defined from the previous cells
# # engine = create_engine(DB_URL)

# print(f"Attempting to drop tables: {', '.join(tables_to_drop)}...")


# # Method 2: Using raw SQL (more direct and safer if metadata is complex)
# try:
#     with engine.connect() as connection:
#         with connection.begin(): # Start a transaction
#             for table_name in tables_to_drop:
#                 print(f"Dropping table {table_name} if it exists...")
#                 # Use 'CASCADE' if there are foreign key dependencies
#                 connection.execute(text(f"DROP TABLE IF EXISTS {table_name} CASCADE"))
#         print(f"✅ Successfully dropped tables: {', '.join(tables_to_drop)}")
# except Exception as e:
#     print(f"❌ Error dropping tables: {e}")


Attempting to drop tables: flights...
2025-05-07 18:08:48,100 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-07 18:08:48,101 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-07 18:08:48,102 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-07 18:08:48,102 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-07 18:08:48,104 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-07 18:08:48,104 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-07 18:08:48,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
Dropping table flights if it exists...
2025-05-07 18:08:48,105 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS flights CASCADE
2025-05-07 18:08:48,105 INFO sqlalchemy.engine.Engine [generated in 0.00016s] {}
2025-05-07 18:08:48,111 INFO sqlalchemy.engine.Engine COMMIT
✅ Successfully dropped tables: flights


In [4]:
# Cell 2: Create tables in the database
def create_tables():
    """
    Create all tables in the PostgreSQL database
    """
    print("Creating tables in PostgreSQL...")
    
    # This will create all tables defined with the Base class
    Base.metadata.create_all(engine)
    print("✅ All tables created successfully!")
    
    # Check if tables exist by querying table names
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    
    print("\nTables in the database:")
    for table in tables:
        print(f"- {table}")

# Execute the function to create tables
create_tables()

Creating tables in PostgreSQL...
2025-05-07 18:08:48,928 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-07 18:08:48,932 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-07 18:08:48,933 INFO sqlalchemy.engine.Engine [generated in 0.00059s] {'table_name': 'flights', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-07 18:08:48,936 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.re

In [6]:
# Cell: Visual representation of table structures
def display_table_schema(table_name):
    """
    Display a visual representation of a table schema
    
    Args:
        table_name: Name of the table to inspect
    """
    # Get table information using SQLAlchemy's inspect
    inspector = inspect(engine)
    columns = inspector.get_columns(table_name)
    pk_constraint = inspector.get_pk_constraint(table_name)
    unique_constraints = inspector.get_unique_constraints(table_name)
    
    # Format column information
    col_info = []
    for col in columns:
        pk_marker = "🔑 " if col['name'] in pk_constraint.get('constrained_columns', []) else ""
        nullable = "NULL" if col['nullable'] else "NOT NULL"
        default = f"DEFAULT {col['default']}" if col['default'] is not None else ""
        
        # Format type with length for string types
        type_str = str(col['type'])
        
        col_info.append({
            "Column": f"{pk_marker}{col['name']}",
            "Type": type_str,
            "Constraints": f"{nullable} {default}".strip()
        })
    
    # Create DataFrame for pretty display
    schema_df = pd.DataFrame(col_info)
    
    print(f"=== Table: {table_name} ===")
    print(schema_df)
    
    # Display unique constraints
    if unique_constraints:
        print("\nUnique Constraints:")
        for uc in unique_constraints:
            print(f"- {uc['name']}: {', '.join(uc['column_names'])}")
    
    print("\n" + "="*50 + "\n")

# Display schema for each table
tables = ["routes", "flights"]
for table in tables:
    display_table_schema(table)

2025-05-06 23:03:33,614 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 23:03:33,615 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_attribute.attname AS name, pg_catalog.format_type(pg_catalog.pg_attribute.atttypid, pg_catalog.pg_attribute.atttypmod) AS format_type, (SELECT pg_catalog.pg_get_expr(pg_catalog.pg_attrdef.adbin, pg_catalog.pg_attrdef.adrelid) AS pg_get_expr_1 
FROM pg_catalog.pg_attrdef 
WHERE pg_catalog.pg_attrdef.adrelid = pg_catalog.pg_attribute.attrelid AND pg_catalog.pg_attrdef.adnum = pg_catalog.pg_attribute.attnum AND pg_catalog.pg_attribute.atthasdef) AS "default", pg_catalog.pg_attribute.attnotnull AS not_null, pg_catalog.pg_class.relname AS table_name, pg_catalog.pg_description.description AS comment, pg_catalog.pg_attribute.attgenerated AS generated, (SELECT json_build_object(%(json_build_object_2)s, pg_catalog.pg_attribute.attidentity = %(attidentity_1)s, %(json_build_object_3)s, pg_catalog.pg_sequence.seqstart, %(json_build_object_4)s, pg_catalog